In [1]:
import datetime
import numpy as np
import os
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt

In [3]:
# load latest constraints that worked with the maximum entropy 
on_mac = True
on_thinkpad = False
on_hpg = False

if on_mac == True:
    data_path = "/Volumes/hodaakl/"
if on_thinkpad== True:
    data_path = "//exasmb.rc.ufl.edu/blue/pdixit/hodaakl/"
if on_hpg == True:
    data_path = "/blue/pdixit/hodaakl/"

# specify the project you are working on     
spec_folder_onServer = data_path + 'A5MCMC_IGF_FoxO/'
# path = spec_folder_onServer + '0107_MuS/'
# ------------------------------------------

# file_name_lambda = path + 'Lambdas.csv'
# file_name_error = path+ 'Errors.csv'
read_dictionary = np.load(spec_folder_onServer + 'Arrays_for_max_ent/Cons_1213_MuS.npy',allow_pickle='TRUE').item()
real_cons = read_dictionary['array']

In [4]:
real_cons

array([4.73326672e+02, 4.54108606e+02, 4.54927976e+02, 4.36524779e+02,
       4.10898456e+02, 3.99717094e+02, 3.96371614e+02, 4.73326672e+02,
       4.63365604e+02, 4.48820850e+02, 4.08610485e+02, 3.74722149e+02,
       3.59573891e+02, 3.57008177e+02, 4.73326672e+02, 4.40846141e+02,
       4.31311828e+02, 3.80035497e+02, 3.30084268e+02, 3.17888074e+02,
       3.16343356e+02, 4.73326672e+02, 4.32454721e+02, 3.89352065e+02,
       2.73958268e+02, 2.19506117e+02, 2.16909031e+02, 2.21131886e+02,
       2.28865070e+05, 2.10509338e+05, 2.11701237e+05, 1.95448774e+05,
       1.74203740e+05, 1.65509616e+05, 1.62740579e+05, 2.29008297e+05,
       2.19578284e+05, 2.06260524e+05, 1.71882226e+05, 1.46135878e+05,
       1.35316891e+05, 1.33540269e+05, 2.28658597e+05, 1.98284244e+05,
       1.90315679e+05, 1.49111430e+05, 1.14298312e+05, 1.06619137e+05,
       1.05796707e+05, 2.30436021e+05, 1.92391258e+05, 1.56617420e+05,
       7.91837397e+04, 5.14056114e+04, 5.02260546e+04, 5.23042157e+04])

In [5]:
# load constraints from the data and save all the means and 2nd moments, then save all the required points 
## saving Data of IGF
filepath  = 'Data/IGF/EXP129_RAW.xlsx'
xls = pd.ExcelFile(filepath)
Data_Dict = defaultdict(lambda: 'Not present')
Sheet_labels = ['Sheet2' ,'0pM','10pM','15pm','20pM','25pM', '50pM', '250pM']
# conc
nSheets = len(Sheet_labels)
for i in range(nSheets): 
    sl = Sheet_labels[i]
    df = pd.read_excel(xls, sl ,header=None)
    data = df.to_numpy()
    Data_Dict[sl] = data
    print(f'saved {sl} to dictionary')

saved Sheet2 to dictionary
saved 0pM to dictionary
saved 10pM to dictionary
saved 15pm to dictionary
saved 20pM to dictionary
saved 25pM to dictionary
saved 50pM to dictionary
saved 250pM to dictionary


In [6]:
df = pd.read_excel(xls, 'Sheet2' ,header=None)
bg_array = df.to_numpy()[:,1]
# bg_matrix = np.transpose(np.tile(bg_array, (300,1)))

conc_labels = ['10pM','15pm','20pM','50pM', '250pM']
# conc
bgg = 0 
ncc = 0
Means_dict = defaultdict(lambda: 'Not present')
Var_dict = defaultdict(lambda:'Not present')
SecondMoment_dict = defaultdict(lambda:'Not present')
for key in conc_labels: 
    print(key)
    matrix = Data_Dict[key][:,1:]
    ncells = matrix.shape[1]
    bg_matrix = np.transpose(np.tile(bg_array, (ncells,1)))
#     print(matrix.shape)
    ## subtract 
    data = matrix - bg_matrix
#     if key 
    bgg += np.sum(data[0,:])
    ncc += len(data[0,:]) 
#     Means_dict[key] = np.mean(data, axis = 1)
#     SecondMoment_dict[key] = np.mean(data**2, axis = 1)
#     Var_dict[key] = np.var(data, axis = 1)
    
flour_noligand = bgg/ncc
print(flour_noligand)

10pM
15pm
20pM
50pM
250pM
22354.1407661559


In [7]:
n_foxO = 710*2/3 
au_foxO = flour_noligand
# scale factor 
sc = n_foxO/au_foxO

### Removing the background and getting the shift

In [9]:
df = pd.read_excel(xls, 'Sheet2' ,header=None)
bg_array = df.to_numpy()[:,1]
# bg_matrix = np.transpose(np.tile(bg_array, (300,1)))

conc_labels = ['10pM','15pm','20pM','50pM', '250pM']
# conc
bgg = 0 
ncc = 0
Means_dict = defaultdict(lambda: 'Not present')
Var_dict = defaultdict(lambda:'Not present')
SecondMoment_dict = defaultdict(lambda:'Not present')
for key in conc_labels: 
    print(key)
    matrix = Data_Dict[key][:,1:]
    ncells = matrix.shape[1]
    bg_matrix = np.transpose(np.tile(bg_array, (ncells,1)))
#     print(matrix.shape)
    ## subtract 
    data = (matrix - bg_matrix)*sc
#     if key 
#     bgg += np.sum(data[0,:])
#     ncc += len(data[0,:]) 
    Means_dict[key] = np.mean(data, axis = 1)
    SecondMoment_dict[key] = np.mean(data**2, axis = 1)
    Var_dict[key] = np.var(data, axis = 1)
    
# bgg/ncc




10pM
15pm
20pM
50pM
250pM


In [10]:
means_matrix = np.zeros((len(conc_labels), 61))
i=0
for key in conc_labels:
    means_matrix[i,:] = Means_dict[key]
    i+=1 
np.shape(means_matrix)
    

(5, 61)

In [11]:
data_shift_dict = defaultdict(lambda: 'not present')
data_shift = (np.average(means_matrix[:,0]) - means_matrix[:,0])
for i in range(np.shape(means_matrix)[0]):
    
    means_matrix[i] = means_matrix[i] + data_shift[i]
    

### Saving the data with bg removal, shift added and scaling .


In [12]:
df = pd.read_excel(xls, 'Sheet2' ,header=None)
bg_array = df.to_numpy()[:,1]
# bg_matrix = np.transpose(np.tile(bg_array, (300,1)))

conc_labels = ['10pM','15pm','20pM','50pM', '250pM']
# conc
bgg = 0 
ncc = 0
Means_dict = defaultdict(lambda: 'Not present')
Var_dict = defaultdict(lambda:'Not present')
SecondMoment_dict = defaultdict(lambda:'Not present')
i=0
for key in conc_labels: 
    print(key)
    matrix = Data_Dict[key][:,1:]
    ncells = matrix.shape[1]
    bg_matrix = np.transpose(np.tile(bg_array, (ncells,1)))
    
#     print((matrix - bg_matrix)*sc )
    
    data = ((matrix - bg_matrix)*sc ) + data_shift[i]
    print(data.shape)
    Means_dict[key] = np.mean(data, axis = 1)
    SecondMoment_dict[key] = np.mean(data**2, axis = 1)
    Var_dict[key] = np.var(data, axis = 1)
    
    i+=1
    
# bgg/ncc




10pM
(61, 301)
15pm
(61, 300)
20pM
(61, 300)
50pM
(61, 300)
250pM
(61, 300)


### Getting the contraints and saving them 

In [14]:
time_values = [0,6,12,24,45,60,90]
times_constrained = np.array(time_values) #minutes 
idx_list = np.array(times_constrained/3, dtype=int)

In [15]:
# constraints_means_dict = defaultdict(lambda: 'not present')
conc_list = [10,15,20,50, 250]
conc_labels = ['10pM','15pm','20pM','50pM', '250pM']
labels_arr_means = []
labels_arr_sm = []
cons_arr = np.zeros(len(conc_list)* len(time_values)*2)
k=0
titles_list = []
means_list = []
sm_list = []
for i in range(len(conc_list)):
    for j in range(len(time_values)):
        title = f'{conc_list[i]}_pM_{time_values[j]}_min'
        # save the means
        key = conc_labels[i]
        means_list.append(Means_dict[key][idx_list[j]])
        # save the second moments 
        sm_list.append(SecondMoment_dict[key][idx_list[j]])
        # save the titles 
        labels_arr_means.append(f'Means_{title}')
        labels_arr_sm.append(f'sm_{title}')



In [19]:
means_arr = np.asarray(means_list)
sm_arr = np.asarray(sm_list)

## 
cons_arr = np.concatenate((means_arr, sm_arr), axis = 0 )
labels_arr = labels_arr_means + labels_arr_sm
### 


print(means_arr)
print(sm_arr)
print(labels_arr)
print(cons_arr)

[473.32546653 453.95539421 454.78124476 436.23248704 410.40347098
 399.13366949 395.76172798 473.32546653 463.28561062 448.62581436
 408.09740328 373.94102539 358.67295192 356.08694447 473.32546653
 440.58802905 430.97830388 379.29639967 328.95007969 316.65741918
 315.10048282 473.32546653 432.13023634 388.68665824 272.3801483
 217.49730464 214.8796769  219.13593348 473.32546653 409.38183645
 303.53751901 198.119894   191.28937667 189.31085995 187.60775032]
[228940.58764282 210438.41850867 211643.06187714 195271.41315022
 173882.43174952 165134.64224684 162346.88394029 229086.08937062
 219581.51099657 206162.04554114 171541.32103033 145641.90960601
 134765.45768148 132979.98907962 228730.83558703 198119.29122717
 190096.14953721 148624.60677435 113635.69350336 105926.63288596
 105102.80199993 230536.48865845 192196.06455159 156179.47151957
  78387.15291841  50578.9350503   49400.25100993  51479.5373443
 228628.7673577  171243.4460985   94812.25410163  41139.4503615
  38546.65669444  37

In [20]:
dictionary = {'labels': labels_arr, 'array':cons_arr}

In [21]:
np.save('/Volumes/hodaakl/A5MCMC_IGF_FoxO/Arrays_for_max_ent/Cons_0114_35Cond_MuS.npy', dictionary) 



In [28]:
## load the dict
loaded_dict = np.load(spec_folder_onServer + 'Arrays_for_max_ent/Cons_0114_35Cond_MuS.npy',allow_pickle='TRUE').item()
cons_arr = loaded_dict['array']
l_arr = loaded_dict['labels']

In [26]:
len(cons_arr)

70

In [29]:
l_arr

['Means_10_pM_0_min',
 'Means_10_pM_6_min',
 'Means_10_pM_12_min',
 'Means_10_pM_24_min',
 'Means_10_pM_45_min',
 'Means_10_pM_60_min',
 'Means_10_pM_90_min',
 'Means_15_pM_0_min',
 'Means_15_pM_6_min',
 'Means_15_pM_12_min',
 'Means_15_pM_24_min',
 'Means_15_pM_45_min',
 'Means_15_pM_60_min',
 'Means_15_pM_90_min',
 'Means_20_pM_0_min',
 'Means_20_pM_6_min',
 'Means_20_pM_12_min',
 'Means_20_pM_24_min',
 'Means_20_pM_45_min',
 'Means_20_pM_60_min',
 'Means_20_pM_90_min',
 'Means_50_pM_0_min',
 'Means_50_pM_6_min',
 'Means_50_pM_12_min',
 'Means_50_pM_24_min',
 'Means_50_pM_45_min',
 'Means_50_pM_60_min',
 'Means_50_pM_90_min',
 'Means_250_pM_0_min',
 'Means_250_pM_6_min',
 'Means_250_pM_12_min',
 'Means_250_pM_24_min',
 'Means_250_pM_45_min',
 'Means_250_pM_60_min',
 'Means_250_pM_90_min',
 'sm_10_pM_0_min',
 'sm_10_pM_6_min',
 'sm_10_pM_12_min',
 'sm_10_pM_24_min',
 'sm_10_pM_45_min',
 'sm_10_pM_60_min',
 'sm_10_pM_90_min',
 'sm_15_pM_0_min',
 'sm_15_pM_6_min',
 'sm_15_pM_12_min',
 '